<a href="https://colab.research.google.com/github/mrjw717/JW-Daily-Text/blob/main/Text%20Category%20Hierarchy%20to%20JSON%20Processor%20with%20Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example Format of Hirachy

```
-Category
--Sub Category 1
---Further Nested Category 1
---Further Nested Category 2
--Sub Category 2
---Sub Category 1
---Sub Category 2
--Sub Category 3
```



In [2]:
import json
import re
from google.colab import files
from collections import defaultdict
import zipfile  # Changed from shutil to zipfile
import os

# Helper function to create nested dictionaries
def build_tree(lines):
    root = {}
    stack = []

    for line in lines:
        if not line.strip():
            continue  # Skip empty lines

        # Calculate the level by counting leading dashes
        level = len(re.match(r"^-*", line).group()) - 1
        name = line.strip("-").strip()

        # Pop the stack to maintain the correct hierarchy
        while stack and len(stack) > level:
            stack.pop()

        # Find the parent for the current node
        if stack:
            parent = stack[-1]
            if name not in parent:
                parent[name] = {}
            stack.append(parent[name])
        else:
            # Top-level category
            if name not in root:
                root[name] = {}
            stack.append(root[name])

    return root

# Create a simple input interface for text entry
from IPython.display import display
import ipywidgets as widgets

text_area = widgets.Textarea(
    value='',
    placeholder='Paste your category hierarchy here...',
    description='Input:',
    layout=widgets.Layout(width='100%', height='200px')
)

process_button = widgets.Button(description='Process')
download_button = widgets.Button(description='Download All JSONs', disabled=True)
output = widgets.Output()

json_files = []  # Store JSON file names globally for download

def on_process_button_click(b):
    global json_files
    with output:
        output.clear_output()
        text_input = text_area.value
        lines = text_input.strip().split("\n")
        category_tree = build_tree(lines)

        # Ensure output directory exists
        os.makedirs("json_output", exist_ok=True)

        # Save each main category as a separate JSON file
        json_files = []
        for main_category, sub_tree in category_tree.items():
            filename = f"json_output/{main_category.replace(' ', '_')}.json"
            with open(filename, "w") as json_file:
                json.dump({main_category: sub_tree}, json_file, indent=2)
            json_files.append(filename)

            # Show JSON structure for each category
            print(f"{main_category} JSON Structure:")
            print(json.dumps({main_category: sub_tree}, indent=2))
            print("\n")

        if json_files:
            download_button.disabled = False
            print("JSON files generated successfully.")
        else:
            print("No JSON files were generated.")

def on_download_button_click(b):
    if not json_files:
        print("No files to download.")
        return

    # Create a zip file of all JSONs
    zip_filename = "json_files.zip"
    with zipfile.ZipFile(zip_filename, 'w') as zipf:  # Changed from shutil.ZipFile to zipfile.ZipFile
        for file in json_files:
            zipf.write(file)

    # Provide the download
    files.download(zip_filename)

process_button.on_click(on_process_button_click)
download_button.on_click(on_download_button_click)

# Display the input box and buttons
display(text_area, process_button, download_button, output)

Textarea(value='', description='Input:', layout=Layout(height='200px', width='100%'), placeholder='Paste your …

Button(description='Process', style=ButtonStyle())

Button(description='Download All JSONs', disabled=True, style=ButtonStyle())

Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>